# 参数管理

* 访问参数，用于调试、诊断和可视化；
* 参数初始化；
* 在不同模型组件间共享参数。

(**首先看一下具有单隐藏层的多层感知机。**)

In [9]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[ 0.0135],
        [-0.0756]], grad_fn=<AddmmBackward0>)

## 参数访问
从已有模型中访问参数。当通过`Sequential`类定义模型时，可通过索引来访问模型的任意层。  
参数是复合的对象，包括值、梯度和额外信息

### 目标参数

In [10]:
print(net[2].state_dict())

print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

net[2].weight.grad == None

OrderedDict([('weight', tensor([[ 0.0475, -0.0731, -0.0513,  0.1706,  0.3019, -0.2245,  0.1473, -0.2425]])), ('bias', tensor([-0.0864]))])
<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0864], requires_grad=True)
tensor([-0.0864])


True

### 一次性访问所有参数

在Python中，星号 * 用于解压可迭代对象，例如列表或元组。当它放置在一个可迭代对象前面时，它会将该对象解压成单独的元素。  
在如下代码中，* 用于解压列表推导式生成的元组列表，以便将其作为单独的参数传递给 print 函数。这样做可以使 print 函数按照我们希望的方式打印元组中的元素，而不是将整个列表作为一个参数打印出来。

In [11]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [12]:
# 另一种访问网络参数的方式
print(net.state_dict())
net.state_dict()['2.bias'].data

OrderedDict([('0.weight', tensor([[ 0.0645,  0.2520, -0.1812,  0.2107],
        [-0.3619, -0.2846, -0.3455,  0.4389],
        [ 0.1730, -0.3560,  0.1068,  0.4658],
        [ 0.0736,  0.0735, -0.1898,  0.3316],
        [ 0.1834,  0.4269, -0.1388, -0.1996],
        [-0.4885, -0.2174,  0.2822, -0.2198],
        [ 0.1989,  0.1240,  0.1747, -0.4638],
        [ 0.4466,  0.0437,  0.3509, -0.3087]])), ('0.bias', tensor([-0.2028, -0.3927, -0.2137, -0.0063,  0.1671, -0.4662, -0.2000, -0.2567])), ('2.weight', tensor([[ 0.0475, -0.0731, -0.0513,  0.1706,  0.3019, -0.2245,  0.1473, -0.2425]])), ('2.bias', tensor([-0.0864]))])


tensor([-0.0864])

### 从嵌套块收集参数
首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。

In [13]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
print(rgnet(X))
print(rgnet)
print(rgnet[0][1][0].bias.data)

tensor([[0.2017],
        [0.2017]], grad_fn=<AddmmBackward0>)
Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)
tensor([ 0.3057,  0.3824, -0.4219, -0.1528,  0.0371, -0.0590, -0.0781,  0.4976])


## 参数初始化

### 内置初始化
apply()函数：对于net的每个layer（module）都调用init_normal()函数，for loop

In [14]:
# 将所有权重参数初始化为标准差为0.01的高斯随机变量， 且将偏置参数设置为0
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0036, -0.0024, -0.0028,  0.0177]), tensor(0.))

In [15]:
# 将所有参数初始化为给定的常数，如初始化为1
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

[**对某些块应用不同的初始化方法**]。
如使用Xavier初始化方法初始化第一个神经网络层；将第三个神经网络层初始化为常量值42

In [16]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0921, -0.2745,  0.5397, -0.2150])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 自定义初始化
如使用以下的分布为任意权重参数$w$定义初始化方法：

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$

In [17]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init)
net[0].weight

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


Parameter containing:
tensor([[-5.2154, -9.9289, -0.0000, -0.0000],
        [-5.0505,  6.4706,  7.3130, -8.4020],
        [ 9.7005, -0.0000,  8.9630, -0.0000],
        [ 0.0000, -9.7680, -9.3312,  8.9540],
        [-0.0000,  0.0000,  8.5914,  5.5630],
        [ 5.5487, -7.6333, -0.0000,  0.0000],
        [-0.0000, -5.3803,  8.9434,  8.3106],
        [ 0.0000,  9.2784,  0.0000,  0.0000]], requires_grad=True)

In [18]:
# 注意，参数可以直接设置
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -8.9289,  1.0000,  1.0000])

## 参数绑定
有时希望在多个层间共享参数：可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。

In [19]:
# 需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


有一个问题：当参数绑定时，梯度会发生什么情况？
答案是由于模型参数包含梯度，因此在反向传播期间第二个隐藏层（即第三个神经网络层）和第三个隐藏层（即第五个神经网络层）的梯度会加在一起。

#### 语法总结：  
* 在Python中，星号 * 用于解压可迭代对象，例如列表或元组。当它放置在一个可迭代对象前面时，它会将该对象解压成单独的元素。  
在如下代码中，* 用于解压列表推导式生成的元组列表，以便将其作为单独的参数传递给 print 函数。这样做可以使 print 函数按照我们希望的方式打印元组中的元素，而不是将整个列表作为一个参数打印出来。  
**print(*[(name, param.shape) for name, param in net[0].named_parameters()])**
* apply()函数：对于net的每个layer（module）都调用init_normal()函数，for loop。  
**net.apply(init_constant)**
* f:一个字符串格式化方法，用于创建带有变量的字符串。  
  具体来说，f'block {i}' 使用了 f-string 的语法，其中 {i} 将被变量 i 的值替换，生成形如 'block 0'、'block 1'、'block    2'、'block 3' 的字符串。这样的字符串用作子模块的名称。  
**for i in range(4):  
    net.add_module(f'block {i}', block1())  
    return net** 
* 在 PyTorch 中，state_dict() 和 named_parameters() 是用于处理模型参数的两个常用方法。  
  **state_dict()** 方法用于返回模型的状态字典。状态字典是一个字典对象，其中包含了模型所有参数的键值对。键是参数的名称，值是参数的张量。这个方法通常用于保存和加载模型，以及在训练过程中保存和恢复模型的状态。  
  **named_parameters()** 方法返回一个生成器，它可以用于遍历模型中的所有参数。每个参数都以 (name, parameter) 的形式返回，其中 name 是参数的名称，parameter 是参数的张量。这个方法通常用于在训练过程中对模型参数进行操作，比如对不同的参数应用不同的学习率。  
  它们的差异主要体现在3方面：
  * 返回值类型不同
  * 存储的模型参数的种类不同
  * 返回的值的require_grad属性不同  
  1.model.state_dict()是将layer_name:layer_param的键值信息存储为dict形式，而model.named_parameters()则是打包成一个元祖然后再存到list当中  
  2.model.state_dict()存储的是该model中包含的所有layer中的所有参数；而model.named_parameters()则只保存可学习、可被更新的参数， model.buffer()中的参数不包含在model.named_parameters()中  
  3.model.state_dict()所存储的模型参数tensor的require_grad属性都是False，而model.named_parameters()的require_grad属性都是True